# Validity testing

- Models: ensemble average, reliability average, linear regression, delta method, NNGP, CNN
- Depths: l2 depth, linf depth, neighbor depth, 
- Shift: quantile corrected and uncorrected
- Variables: WN, TAS, TASMAX, PR

In [1]:
import os
import gc
import math

# numpy
import numpy as np
import scipy
np.set_printoptions(suppress=True)

# from jax
import jax
from jax import jacfwd, jacrev
from jax import vmap, grad, jit, random
from jax.config import config
import jax.numpy as jnp
from jax.tree_util import tree_map, tree_flatten, tree_unflatten, tree_leaves

import haiku as hk
import flax
from flax import linen as nn
import optax

import properscoring as ps

config.update("jax_enable_x64", True)

# plotting
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.ticker import FormatStrFormatter
plt.style.use('default')

from matplotlib.gridspec import GridSpec
from matplotlib import colors
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

from tqdm.notebook import tqdm
from tqdm.notebook import trange

# netCDF
import netCDF4 as nc
import pickle
import warnings
warnings.filterwarnings('ignore')

from sklearn.decomposition import PCA

In [2]:
def moving_average(a, n=12):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def mse_weighted(x, y):
    nlat = x.shape[1]
    weight = np.cos(np.linspace(math.pi/2 - 1/nlat, -math.pi/2 + 1/nlat, nlat))
    weight /= np.mean(weight)
    weight = weight[None,:,None]
    
    return np.sqrt(np.mean((x - y)**2 * weight, axis = (1, 2)))

def resize_model(x, dim1, dim2): 
    x = np.moveaxis(x, 0, 2)
    x = resize(x, (dim1, dim2))
    x = np.moveaxis(x, 2, 0)
    return x

def sliced_wasserstein(x, y, w):
    x = x @ w
    y = y @ w
    
    qx = jnp.quantile(x, q = jnp.linspace(0, 1, 100), axis = 0)
    qy = jnp.quantile(y, q = jnp.linspace(0, 1, 100), axis = 0)
    return jnp.mean(jnp.sqrt(jnp.mean((qx - qy)**2, axis = 0)))

sliced_wasserstein = jit(sliced_wasserstein)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def logit(x):
    return np.log(x/(1-x))

def softplus(x):
    return np.log(1 + np.exp(x))

def inv_softplus(x):
    return np.log(np.exp(x) - 1)

In [3]:
def width_weighted(x):
    nlat = x.shape[1]
    weight = jnp.cos(np.linspace(math.pi/2 - 1/nlat, -math.pi/2 + 1/nlat, nlat))
    weight2 = weight/jnp.mean(weight)
    weight2 = weight2[:,None]
    
    width = jnp.max(x, axis = 0) - jnp.min(x, axis = 0)
    return jnp.mean(width * weight2)

width_weighted = jit(width_weighted)

In [4]:
def scale_and_split(xtrain, xtest, model_no, nval = 200, reshape = False):
    ytrain = xtrain[:,model_no]
    ytest = xtest[:,model_no]
    xtrain = np.delete(xtrain, model_no, axis=1)
    xtest = np.delete(xtest, model_no, axis=1)

    ## dimensions
    ntrain = xtrain.shape[0]
    ntest = xtest.shape[0]
    nlat = xtrain.shape[2]
    nlon = xtrain.shape[3]

    ## rescale
    xtrain_mean = np.mean(xtrain, axis = 1)
    xtest_mean = np.mean(xtest, axis = 1)
    xtrain_sd = np.std(xtrain, axis = 1)
    xtest_sd = np.std(xtest, axis = 1)

    xtrain_scaled = (xtrain - xtrain_mean[:,None]) / xtrain_sd[:,None]
    xtest_scaled = (xtest - xtest_mean[:,None]) / xtest_sd[:,None]
    ytrain_scaled = (ytrain - xtrain_mean) / xtrain_sd
    ytest_scaled = (ytest - xtest_mean) / xtest_sd

    ## convert
    if reshape:
        xtrain_scaled = xtrain_scaled.reshape(ntrain, -1)
        xtest_scaled = xtest_scaled.reshape(ntest, -1)
        ytrain_scaled = ytrain_scaled.reshape(ntrain, -1)
        ytest_scaled = ytest_scaled.reshape(ntest, -1)

    xtrain_scaled = jnp.array(xtrain_scaled)
    xtest_scaled = jnp.array(xtest_scaled)
    ytrain_scaled = jnp.array(ytrain_scaled)
    ytest_scaled = jnp.array(ytest_scaled)

    xval_scaled = xtrain_scaled[-nval:]
    yval_scaled = ytrain_scaled[-nval:]
    xtrain_scaled = xtrain_scaled[:-nval]
    ytrain_scaled = ytrain_scaled[:-nval]
    
    return xtrain_scaled, xval_scaled, xtest_scaled, ytrain_scaled, yval_scaled, ytest_scaled

## depths

In [5]:
def inf_depth(y):
    dist = jnp.max(jnp.abs(y), axis = (1, 2))
    return 1/(1 + dist)
inf_depth = jit(inf_depth)

def global_mmd(x, y):
    dist = jnp.max(jnp.abs(x - y), axis = (1, 2))
    dist = jnp.mean(dist)
    return 1/(1 + dist)
global_mmd = jit(vmap(global_mmd, (None, 0)))

def global_mmd_self(x, y):
    dist = jnp.max(jnp.abs(x - y), axis = (1, 2))
    dist = jnp.mean(jnp.sort(dist)[1:])
    return 1/(1 + dist)
global_mmd_self = jit(vmap(global_mmd_self, (None, 0)))

# def tukey_depth(x, y):
#     fx = jnp.mean(x < y, axis = 0)
#     depth = 1 - jnp.abs(1 - 2 * fx)
#     return jnp.mean(depth)
# tukey_depth = jit(vmap(tukey_depth, (None, 0)))

In [6]:
def field_min(x, y):
    return jnp.min(jnp.array([x, y]))
field_min = jit(vmap(vmap(field_min, (0, 0)), (1, 1)))

def tukey_depth(x, y):
    fx_left = jnp.mean(x < y, axis = 0)
    fx_right = jnp.mean(x > y, axis = 0)
    return jnp.mean(2 * field_min(fx_left, fx_right))
tukey_depth = jit(vmap(tukey_depth, (None, 0)))

def tukey_depth_self(x, y):
    fx_left = jnp.sum(x < y, axis = 0)/(x.shape[0] - 1)
    fx_right = jnp.sum(x > y, axis = 0)/(x.shape[0] - 1)
    return jnp.mean(2 * field_min(fx_left, fx_right))
tukey_depth_self = jit(vmap(tukey_depth_self, (None, 0)))

In [7]:
def field_width(x):
    lower = jnp.min(x, axis = 0)
    upper = jnp.max(x, axis = 0)
    lower = jnp.clip(lower, -5, 5)
    upper = jnp.clip(upper, -5, 5)
    return jnp.mean(upper - lower)
field_width = jit(field_width)

## White noise control run

In [8]:
nproj = 6
nval = 200

nmod = 30
ntrain = 1000
nval = 200
ntest = 1000
np.random.seed(0)
ztrain = np.random.normal(0, 1, [ntrain, nmod, 30, 50])
ztest = np.random.normal(0, 1, [ntest, nmod, 30, 50])
ntrain = ntrain - nval

save_loc = 'trained_models/wn'

In [9]:
analysis = ['ens', 'wa', 'delta', 'lm', 'nngp', 'cnn1']
alpha = 0.1

cover1 = np.zeros((nmod, nproj))
cover2 = np.zeros((nmod, nproj))
cover3 = np.zeros((nmod, nproj))
width1 = np.zeros((nmod, nproj))
width2 = np.zeros((nmod, nproj))
width3 = np.zeros((nmod, nproj))
level = np.ceil((1 - alpha) * (nval + 1))/(nval + 1)

for model_no in trange(nmod):
    xtrain, xval, xtest, ytrain, yval, ytest = scale_and_split(ztrain, ztest, model_no = model_no, nval = nval)
    
    for k in trange(nproj, leave = False):
        
        yval_hat = np.load(save_loc + f'/yval_{analysis[k]}_{model_no}.npy')
        ytest_hat = np.load(save_loc + f'/ytest_{analysis[k]}_{model_no}.npy')
        
        if analysis[k] == 'lm':
            yval_hat = np.clip(yval_hat, -3, 3)
            ytest_hat = np.clip(ytest_hat, -3, 3)
        
        resval = yval - yval_hat.reshape(yval.shape)
        restest = ytest - ytest_hat.reshape(ytest.shape)
        
        dr1a, dr1b = global_mmd_self(resval, resval), global_mmd(resval, restest)
        dr2a, dr2b = inf_depth(resval), inf_depth(restest)
        dr3a, dr3b = tukey_depth_self(resval, resval), tukey_depth(resval, restest)
        
        q1 = np.sort(dr1a)[int(np.ceil((1 - alpha) * (nval + 1)))]
        q2 = np.sort(dr2a)[int(np.ceil((1 - alpha) * (nval + 1)))]
        q3 = np.sort(dr3a)[int(np.ceil((1 - alpha) * (nval + 1)))]
        
        q1 = (q1 + np.quantile(dr1a, level))/2
        q2 = (q2 + np.quantile(dr2a, level))/2
        q3 = (q3 + np.quantile(dr3a, level))/2
        
        cover1[model_no,k] = np.mean(dr1b < q1)
        cover2[model_no,k] = np.mean(dr2b < q2)
        cover3[model_no,k] = np.mean(dr3b < q3)
        
        width1[model_no,k] = field_width(resval[dr1a < q1])
        width2[model_no,k] = field_width(resval[dr2a < q2])
        width3[model_no,k] = field_width(resval[dr3a < q3])

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [10]:
wn_cover1 = np.mean(cover1, axis = 0)
wn_cover2 = np.mean(cover2, axis = 0)
wn_cover3 = np.mean(cover3, axis = 0)
wn_width1 = np.mean(width1, axis = 0)
wn_width2 = np.mean(width2, axis = 0)
wn_width3 = np.mean(width3, axis = 0)

In [11]:
np.round(wn_cover1, 3),  np.round(wn_cover3, 3), np.round(wn_cover2, 3)

(array([0.902, 0.9  , 0.902, 0.902, 0.902, 0.9  ]),
 array([0.9  , 0.9  , 0.9  , 0.903, 0.9  , 0.905]),
 array([0.901, 0.901, 0.9  , 0.901, 0.9  , 0.894]))

In [12]:
np.round(wn_cover1, 3),  np.round(wn_cover3, 3), np.round(wn_cover2, 3)

(array([0.902, 0.9  , 0.902, 0.902, 0.902, 0.9  ]),
 array([0.9  , 0.9  , 0.9  , 0.903, 0.9  , 0.905]),
 array([0.901, 0.901, 0.9  , 0.901, 0.9  , 0.894]))

In [13]:
np.round(wn_width1, 3), np.round(wn_width3, 3), np.round(wn_width2, 3)

(array([6.126, 6.22 , 6.126, 6.323, 6.126, 6.353]),
 array([6.1  , 6.195, 6.1  , 6.33 , 6.1  , 6.326]),
 array([6.12 , 6.213, 6.12 , 6.332, 6.12 , 6.347]))

## Temperature (TAS)

In [14]:
nproj = 6
nval = 200

xhist_tas = pickle.load(open('../data/xhist_tas_hr.pkl', 'rb'))
xrcp_tas = pickle.load(open('../data/xrcp_tas_hr.pkl', 'rb'))
nmod = xhist_tas.shape[1]

n = (2015 - 1940) * 12
xhist_tas = xhist_tas[-n:]

xtrain_orig = np.concatenate([xhist_tas, xrcp_tas[:108]], axis = 0)
xtest_orig = xrcp_tas[108:]

ntrain = xtrain_orig.shape[0] - nval
ntest = xtest_orig.shape[0]

save_loc = 'trained_models/tas'

In [15]:
analysis = ['ens', 'wa', 'delta', 'lm', 'nngp', 'cnn1']
alpha = 0.1

cover1 = np.zeros((nmod, nproj))
cover2 = np.zeros((nmod, nproj))
cover3 = np.zeros((nmod, nproj))
width1 = np.zeros((nmod, nproj))
width2 = np.zeros((nmod, nproj))
width3 = np.zeros((nmod, nproj))
level = np.ceil((1 - alpha) * (nval + 1))/(nval + 1)

for model_no in trange(nmod):
    xtrain, xval, xtest, ytrain, yval, ytest = scale_and_split(xtrain_orig, xtest_orig, model_no = model_no, nval = nval)
    
    for k in trange(nproj, leave = False):
        
        yval_hat = np.load(save_loc + f'/yval_{analysis[k]}_{model_no}.npy')
        ytest_hat = np.load(save_loc + f'/ytest_{analysis[k]}_{model_no}.npy')
        
        if analysis[k] == 'lm':
            yval_hat = np.clip(yval_hat, -3, 3)
            ytest_hat = np.clip(ytest_hat, -3, 3)
        
        resval = yval - yval_hat.reshape(yval.shape)
        restest = ytest - ytest_hat.reshape(ytest.shape)
        
        dr1a, dr1b = global_mmd_self(resval, resval), global_mmd(resval, restest)
        dr2a, dr2b = inf_depth(resval), inf_depth(restest)
        dr3a, dr3b = tukey_depth_self(resval, resval), tukey_depth(resval, restest)
        
        q1 = np.sort(dr1a)[int(np.ceil((1 - alpha) * (nval + 1)))]
        q2 = np.sort(dr2a)[int(np.ceil((1 - alpha) * (nval + 1)))]
        q3 = np.sort(dr3a)[int(np.ceil((1 - alpha) * (nval + 1)))]
        
        q1 = (q1 + np.quantile(dr1a, level))/2
        q2 = (q2 + np.quantile(dr2a, level))/2
        q3 = (q3 + np.quantile(dr3a, level))/2
        
        cover1[model_no,k] = np.mean(dr1b < q1)
        cover2[model_no,k] = np.mean(dr2b < q2)
        cover3[model_no,k] = np.mean(dr3b < q3)
        
        width1[model_no,k] = field_width(resval[dr1a < q1])
        width2[model_no,k] = field_width(resval[dr2a < q2])
        width3[model_no,k] = field_width(resval[dr3a < q3])

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [16]:
tas_cover1 = np.mean(cover1, axis = 0)
tas_cover2 = np.mean(cover2, axis = 0)
tas_cover3 = np.mean(cover3, axis = 0)
tas_width1 = np.mean(width1, axis = 0)
tas_width2 = np.mean(width2, axis = 0)
tas_width3 = np.mean(width3, axis = 0)

In [17]:
np.round(tas_cover1, 3),  np.round(tas_cover3, 3), np.round(tas_cover2, 3)

(array([0.89 , 0.921, 0.89 , 0.9  , 0.932, 0.876]),
 array([0.928, 0.967, 0.928, 0.975, 0.977, 0.936]),
 array([0.87 , 0.933, 0.904, 0.874, 0.931, 0.895]))

In [18]:
np.round(tas_width1, 3), np.round(tas_width3, 3), np.round(tas_width2, 3)

(array([3.717, 3.563, 3.731, 5.341, 3.376, 3.686]),
 array([3.717, 3.563, 3.731, 5.346, 3.375, 3.687]),
 array([3.705, 3.562, 3.73 , 5.328, 3.374, 3.684]))

## Temperature max (tmax)

In [19]:
nproj = 6
nval = 200

xhist_tas = pickle.load(open('../data/xhist_tasmax_hr.pkl', 'rb'))
xrcp_tas = pickle.load(open('../data/xrcp_tasmax_hr.pkl', 'rb'))
nmod = xhist_tas.shape[1]

n = (2015 - 1940) * 12
xhist_tas = xhist_tas[-n:]

xtrain_orig = np.concatenate([xhist_tas, xrcp_tas[:108]], axis = 0)
xtest_orig = xrcp_tas[108:]

ntrain = xtrain_orig.shape[0] - nval
ntest = xtest_orig.shape[0]

save_loc = 'trained_models/tmax'

In [20]:
analysis = ['ens', 'wa', 'delta', 'lm', 'nngp', 'cnn1']
alpha = 0.1

cover1 = np.zeros((nmod, nproj))
cover2 = np.zeros((nmod, nproj))
cover3 = np.zeros((nmod, nproj))
width1 = np.zeros((nmod, nproj))
width2 = np.zeros((nmod, nproj))
width3 = np.zeros((nmod, nproj))
level = np.ceil((1 - alpha) * (nval + 1))/(nval + 1)

for model_no in trange(nmod):
    xtrain, xval, xtest, ytrain, yval, ytest = scale_and_split(xtrain_orig, xtest_orig, model_no = model_no, nval = nval)
    
    for k in trange(nproj, leave = False):
        
        yval_hat = np.load(save_loc + f'/yval_{analysis[k]}_{model_no}.npy')
        ytest_hat = np.load(save_loc + f'/ytest_{analysis[k]}_{model_no}.npy')
        
        if analysis[k] == 'lm':
            yval_hat = np.clip(yval_hat, -3, 3)
            ytest_hat = np.clip(ytest_hat, -3, 3)
        
        resval = yval - yval_hat.reshape(yval.shape)
        restest = ytest - ytest_hat.reshape(ytest.shape)
        
        dr1a, dr1b = global_mmd_self(resval, resval), global_mmd(resval, restest)
        dr2a, dr2b = inf_depth(resval), inf_depth(restest)
        dr3a, dr3b = tukey_depth_self(resval, resval), tukey_depth(resval, restest)
        
        q1 = np.sort(dr1a)[int(np.ceil((1 - alpha) * (nval + 1)))]
        q2 = np.sort(dr2a)[int(np.ceil((1 - alpha) * (nval + 1)))]
        q3 = np.sort(dr3a)[int(np.ceil((1 - alpha) * (nval + 1)))]
        
        q1 = (q1 + np.quantile(dr1a, level))/2
        q2 = (q2 + np.quantile(dr2a, level))/2
        q3 = (q3 + np.quantile(dr3a, level))/2
        
        cover1[model_no,k] = np.mean(dr1b < q1)
        cover2[model_no,k] = np.mean(dr2b < q2)
        cover3[model_no,k] = np.mean(dr3b < q3)
        
        width1[model_no,k] = field_width(resval[dr1a < q1])
        width2[model_no,k] = field_width(resval[dr2a < q2])
        width3[model_no,k] = field_width(resval[dr3a < q3])

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [21]:
tmax_cover1 = np.mean(cover1, axis = 0)
tmax_cover2 = np.mean(cover2, axis = 0)
tmax_cover3 = np.mean(cover3, axis = 0)
tmax_width1 = np.mean(width1, axis = 0)
tmax_width2 = np.mean(width2, axis = 0)
tmax_width3 = np.mean(width3, axis = 0)

In [22]:
np.round(tmax_cover1, 3), np.round(tmax_cover3, 3), np.round(tmax_cover2, 3)

(array([0.893, 0.922, 0.893, 0.913, 0.933, 0.856]),
 array([0.929, 0.964, 0.929, 0.969, 0.975, 0.935]),
 array([0.831, 0.925, 0.893, 0.868, 0.926, 0.882]))

In [23]:
np.round(tmax_width1, 3), np.round(tmax_width3, 3), np.round(tmax_width2, 3)

(array([3.532, 3.474, 3.605, 4.831, 3.295, 3.646]),
 array([3.534, 3.472, 3.605, 4.837, 3.293, 3.644]),
 array([3.524, 3.474, 3.604, 4.816, 3.295, 3.645]))

## Precipitation (pr)

In [24]:
nproj = 6
nval = 200

xhist_tas = pickle.load(open('../data/xhist_pr_hr.pkl', 'rb'))
xrcp_tas = pickle.load(open('../data/xrcp_pr_hr.pkl', 'rb'))
nmod = xhist_tas.shape[1]

n = (2015 - 1940) * 12
xhist_tas = xhist_tas[-n:]

xtrain_orig = np.concatenate([xhist_tas, xrcp_tas[:108]], axis = 0)
xtest_orig = xrcp_tas[108:]

ntrain = xtrain_orig.shape[0] - nval
ntest = xtest_orig.shape[0]

xtrain_orig = np.log(np.abs(xtrain_orig) + 1e-3)
xtest_orig = np.log(np.abs(xtest_orig) + 1e-3)

save_loc = 'trained_models/pr'

In [25]:
analysis = ['ens', 'wa', 'delta', 'lm', 'nngp', 'cnn1']
alpha = 0.1

cover1 = np.zeros((nmod, nproj))
cover2 = np.zeros((nmod, nproj))
cover3 = np.zeros((nmod, nproj))
width1 = np.zeros((nmod, nproj))
width2 = np.zeros((nmod, nproj))
width3 = np.zeros((nmod, nproj))
level = np.ceil((1 - alpha) * (nval + 1))/(nval + 1)

for model_no in trange(nmod):
    xtrain, xval, xtest, ytrain, yval, ytest = scale_and_split(xtrain_orig, xtest_orig, model_no = model_no, nval = nval)
    
    for k in trange(nproj, leave = False):
        
        yval_hat = np.load(save_loc + f'/yval_{analysis[k]}_{model_no}.npy')
        ytest_hat = np.load(save_loc + f'/ytest_{analysis[k]}_{model_no}.npy')
        
        if analysis[k] == 'lm':
            yval_hat = np.clip(yval_hat, -3, 3)
            ytest_hat = np.clip(ytest_hat, -3, 3)
        
        resval = yval - yval_hat.reshape(yval.shape)
        restest = ytest - ytest_hat.reshape(ytest.shape)
        
        dr1a, dr1b = global_mmd_self(resval, resval), global_mmd(resval, restest)
        dr2a, dr2b = inf_depth(resval), inf_depth(restest)
        dr3a, dr3b = tukey_depth_self(resval, resval), tukey_depth(resval, restest)
        
        q1 = np.sort(dr1a)[int(np.ceil((1 - alpha) * (nval + 1)))]
        q2 = np.sort(dr2a)[int(np.ceil((1 - alpha) * (nval + 1)))]
        q3 = np.sort(dr3a)[int(np.ceil((1 - alpha) * (nval + 1)))]
        
        q1 = (q1 + np.quantile(dr1a, level))/2
        q2 = (q2 + np.quantile(dr2a, level))/2
        q3 = (q3 + np.quantile(dr3a, level))/2
        
        cover1[model_no,k] = np.mean(dr1b < q1)
        cover2[model_no,k] = np.mean(dr2b < q2)
        cover3[model_no,k] = np.mean(dr3b < q3)
        
        width1[model_no,k] = field_width(resval[dr1a < q1])
        width2[model_no,k] = field_width(resval[dr2a < q2])
        width3[model_no,k] = field_width(resval[dr3a < q3])

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [26]:
pr_cover1 = np.mean(cover1, axis = 0)
pr_cover2 = np.mean(cover2, axis = 0)
pr_cover3 = np.mean(cover3, axis = 0)
pr_width1 = np.mean(width1, axis = 0)
pr_width2 = np.mean(width2, axis = 0)
pr_width3 = np.mean(width3, axis = 0)


In [27]:
np.round(pr_cover1, 3), np.round(pr_cover3, 3), np.round(pr_cover2, 3)

(array([0.906, 0.91 , 0.906, 0.912, 0.908, 0.901]),
 array([0.929, 0.938, 0.929, 0.945, 0.949, 0.928]),
 array([0.906, 0.92 , 0.906, 0.903, 0.916, 0.909]))

In [28]:
np.round(pr_width1, 3), np.round(pr_width3, 3), np.round(pr_width2, 3)

(array([2.611, 2.884, 2.871, 3.121, 2.805, 2.952]),
 array([2.617, 2.887, 2.877, 3.128, 2.809, 2.958]),
 array([2.612, 2.885, 2.872, 3.119, 2.806, 2.955]))

In [29]:
np.stack([pr_cover1, pr_cover3, pr_cover2]).T

array([[0.90635965, 0.92923977, 0.90566521],
       [0.90950293, 0.9381579 , 0.92035819],
       [0.90635965, 0.92923977, 0.90566521],
       [0.91209796, 0.94510235, 0.90307018],
       [0.90774854, 0.94883042, 0.91578948],
       [0.901462  , 0.92774124, 0.90891814]])

In [30]:
np.stack([pr_cover1, pr_cover2, pr_cover3]).T

array([[0.90635965, 0.90566521, 0.92923977],
       [0.90950293, 0.92035819, 0.9381579 ],
       [0.90635965, 0.90566521, 0.92923977],
       [0.91209796, 0.90307018, 0.94510235],
       [0.90774854, 0.91578948, 0.94883042],
       [0.901462  , 0.90891814, 0.92774124]])

In [31]:
wn_cover = np.round(np.vstack([wn_cover1, wn_cover3, wn_cover2]).T.reshape(-1), 3)
tas_cover = np.round(np.vstack([tas_cover1, tas_cover3, tas_cover2]).T.reshape(-1), 3)
tmax_cover = np.round(np.vstack([tmax_cover1, tmax_cover3, tmax_cover2]).T.reshape(-1), 3)
pr_cover = np.round(np.vstack([pr_cover1, pr_cover3, pr_cover2]).T.reshape(-1), 3)
cover = np.vstack([wn_cover, tas_cover, tmax_cover, pr_cover]).T

wn_width = np.round(np.vstack([wn_width1, wn_width3, wn_width2]).T.reshape(-1), 3)
tas_width = np.round(np.vstack([tas_width1, tas_width3, tas_width2]).T.reshape(-1), 3)
tmax_width = np.round(np.vstack([tmax_width1, tmax_width3, tmax_width2]).T.reshape(-1), 3)
pr_width = np.round(np.vstack([pr_width1, pr_width3, pr_width2]).T.reshape(-1), 3)
width = np.vstack([wn_width, tas_width, tmax_width, pr_width]).T

In [78]:
np.set_printoptions(precision=3, floatmode = 'fixed')
table = np.hstack([cover, width])

In [79]:
for row in table:
    x = [f"{i:1.3f}" for i in row]
    print(' & '.join(x))

0.902 & 0.890 & 0.893 & 0.906 & 6.126 & 3.717 & 3.532 & 2.611
0.900 & 0.928 & 0.929 & 0.929 & 6.100 & 3.717 & 3.534 & 2.617
0.901 & 0.870 & 0.831 & 0.906 & 6.120 & 3.705 & 3.524 & 2.612
0.900 & 0.921 & 0.922 & 0.910 & 6.220 & 3.563 & 3.474 & 2.884
0.900 & 0.967 & 0.964 & 0.938 & 6.195 & 3.563 & 3.472 & 2.887
0.901 & 0.933 & 0.925 & 0.920 & 6.213 & 3.562 & 3.474 & 2.885
0.902 & 0.890 & 0.893 & 0.906 & 6.126 & 3.731 & 3.605 & 2.871
0.900 & 0.928 & 0.929 & 0.929 & 6.100 & 3.731 & 3.605 & 2.877
0.900 & 0.904 & 0.893 & 0.906 & 6.120 & 3.730 & 3.604 & 2.872
0.902 & 0.900 & 0.913 & 0.912 & 6.323 & 5.341 & 4.831 & 3.121
0.903 & 0.975 & 0.969 & 0.945 & 6.330 & 5.346 & 4.837 & 3.128
0.901 & 0.874 & 0.868 & 0.903 & 6.332 & 5.328 & 4.816 & 3.119
0.902 & 0.932 & 0.933 & 0.908 & 6.126 & 3.376 & 3.295 & 2.805
0.900 & 0.977 & 0.975 & 0.949 & 6.100 & 3.375 & 3.293 & 2.809
0.900 & 0.931 & 0.926 & 0.916 & 6.120 & 3.374 & 3.295 & 2.806
0.900 & 0.876 & 0.856 & 0.901 & 6.353 & 3.686 & 3.646 & 2.952
0.905 & 

In [82]:
analysis = ['EA', 'WA', 'Delta', 'LM', 'GP', 'CNN']
for i, row in enumerate(table):
    x = [f"{i:1.3f}" for i in row]
    print(analysis[i//3], '\t', ' & '.join(x))

EA 	 0.902 & 0.890 & 0.893 & 0.906 & 6.126 & 3.717 & 3.532 & 2.611
EA 	 0.900 & 0.928 & 0.929 & 0.929 & 6.100 & 3.717 & 3.534 & 2.617
EA 	 0.901 & 0.870 & 0.831 & 0.906 & 6.120 & 3.705 & 3.524 & 2.612
WA 	 0.900 & 0.921 & 0.922 & 0.910 & 6.220 & 3.563 & 3.474 & 2.884
WA 	 0.900 & 0.967 & 0.964 & 0.938 & 6.195 & 3.563 & 3.472 & 2.887
WA 	 0.901 & 0.933 & 0.925 & 0.920 & 6.213 & 3.562 & 3.474 & 2.885
Delta 	 0.902 & 0.890 & 0.893 & 0.906 & 6.126 & 3.731 & 3.605 & 2.871
Delta 	 0.900 & 0.928 & 0.929 & 0.929 & 6.100 & 3.731 & 3.605 & 2.877
Delta 	 0.900 & 0.904 & 0.893 & 0.906 & 6.120 & 3.730 & 3.604 & 2.872
LM 	 0.902 & 0.900 & 0.913 & 0.912 & 6.323 & 5.341 & 4.831 & 3.121
LM 	 0.903 & 0.975 & 0.969 & 0.945 & 6.330 & 5.346 & 4.837 & 3.128
LM 	 0.901 & 0.874 & 0.868 & 0.903 & 6.332 & 5.328 & 4.816 & 3.119
GP 	 0.902 & 0.932 & 0.933 & 0.908 & 6.126 & 3.376 & 3.295 & 2.805
GP 	 0.900 & 0.977 & 0.975 & 0.949 & 6.100 & 3.375 & 3.293 & 2.809
GP 	 0.900 & 0.931 & 0.926 & 0.916 & 6.120 & 3.374 & 